## Data Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageFile

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Function to remove corrupted images
def removeCorruptedImages(path):
    for filename in os.listdir(path):
        try:
            img = Image.open(os.path.join(path,filename))
            img.verify()
        except (IOError, SyntaxError) as e:
            print('Bad file:', filename)
            os.remove(os.path.join(path,filename))

In [ ]:
basedir = "/content/drive/MyDrive/YOGA/DATASET"
traindir = os.path.join(basedir, 'TRAIN')
validdir = os.path.join(basedir, 'TEST')

# Removing corrupted images
classes = ['downdog', 'goddess', 'plank', 'tree', 'warrior']
for cls in classes:
    removeCorruptedImages(os.path.join(traindir, cls))
    removeCorruptedImages(os.path.join(validdir, cls))

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(traindir, target_size = (256, 256), batch_size = 16, class_mode = 'categorical')
valid_generator = test_datagen.flow_from_directory(validdir, target_size = (256, 256), batch_size = 16, class_mode = 'categorical')

Found 1081 images belonging to 5 classes.
Found 470 images belonging to 5 classes.


## Creating Model

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(weights="imagenet", input_shape=(256, 256, 3), include_top=False)  # Loads pre-trained ImageNet weights

model = Sequential([
    Flatten(input_shape = base_model.output_shape[1:]),
    Dense(128, activation = 'relu'),
    Dropout(0.2),
    Dense(64, activation = 'relu'),
    Dropout(0.2),
    Dense(5, activation = 'softmax')
])

model = Model(inputs = base_model.input, outputs = model(base_model.output))

<ipython-input-6-4dd2d5cd7fc6>:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet", input_shape=(256, 256, 3), include_top=False)  # Loads pre-trained ImageNet weights


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Compiling the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 128, 128, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 128, 128, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 128, 128, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 128, 128, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 128, 128, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 128, 128, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 128, 128, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 128, 128, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 128, 128, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 128, 128, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 128, 128, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 129, 129, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 64, 64, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 12,752,453 (48.65 MB)

 Trainable params: 12,718,341 (48.52 MB)

 Non-trainable params: 34,112 (133.25 KB)

## Training

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="accuracy",
    min_delta=0.01,
    patience=3,
    verbose=1,
    mode="max",
    restore_best_weights=True
)

classifier = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=15,
    verbose=True,
    callbacks=[early_stop]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
15/68 ━━━━━━━━━━━━━━━━━━━━ 3:58 5s/step - accuracy: 0.2144 - loss: 2.3848

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


68/68 ━━━━━━━━━━━━━━━━━━━━ 387s 5s/step - accuracy: 0.3927 - loss: 1.7519 - val_accuracy: 0.8489 - val_loss: 0.3741
Epoch 2/15
68/68 ━━━━━━━━━━━━━━━━━━━━ 334s 5s/step - accuracy: 0.7540 - loss: 0.5942 - val_accuracy: 0.9426 - val_loss: 0.1751
Epoch 3/15
68/68 ━━━━━━━━━━━━━━━━━━━━ 353s 5s/step - accuracy: 0.8509 - loss: 0.4083 - val_accuracy: 0.9383 - val_loss: 0.1838
Epoch 4/15
68/68 ━━━━━━━━━━━━━━━━━━━━ 382s 5s/step - accuracy: 0.9346 - loss: 0.2113 - val_accuracy: 0.9468 - val_loss: 0.2093
Epoch 5/15
68/68 ━━━━━━━━━━━━━━━━━━━━ 344s 5s/step - accuracy: 0.9444 - loss: 0.1610 - val_accuracy: 0.9426 - val_loss: 0.2513
Epoch 6/15
68/68 ━━━━━━━━━━━━━━━━━━━━ 354s 5s/step - accuracy: 0.9545 - loss: 0.1129 - val_accuracy: 0.9660 - val_loss: 0.1532
Epoch 7/15
68/68 ━━━━━━━━━━━━━━━━━━━━ 335s 5s/step - accuracy: 0.9697 - loss: 0.0921 - val_accuracy: 0.9681 - val_loss: 0.1243
Epoch 8/15
68/68 ━━━━━━━━━━━━━━━━━━━━ 336s 5s/step - accuracy: 0.9685 - loss: 0.0978 - val_accuracy: 0.9468 - val_loss: 0.

## Evaluation

In [ ]:
# Evaluating the model
val_loss, val_accuracy = model.evaluate(valid_generator)
print(f'Validation loss: {val_loss}')
print(f'Validation accuracy: {val_accuracy}')

30/30 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.9694 - loss: 0.0866
Validation loss: 0.12432684749364853
Validation accuracy: 0.9680851101875305


## Results

In [ ]:
# Saving the model
model.save("/content/drive/MyDrive/YOGA/TRAINED_MODEL/yoga-model.keras")